# Exercise 1: Compute pi on the GPU

0. CPU version: about 18 seconds for 512 000 000 points
1. Maximum 1024 threads (one block with 1024 threads)
2. Slow with 512 000 000 threads, 1.5 seconds (n blocks with 512 threads each)
3. Less memory. 512 000 000 in 0.029076 seconds (shared memory reduction). About factor 50 speedup
4. More work per thread (less memory). 512 000 000 in 0.003880 seconds (for-loop in kernel). About factor 10 speedup (or more!) uses 2.16 seconds for 51 200 000 000 000 darts
5. Prepared call kernel launch => no benefit right now. We run the kernel only once! **<-- We are here**
6. Sum/reduction on the GPU
7. Further profile-driven optimizations

In [38]:
%%time
!pip install pycuda

CPU times: user 20.7 ms, sys: 2.05 ms, total: 22.8 ms
Wall time: 1.91 s


In [39]:
import pycuda
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import pycuda.autoinit
import numpy as np
import time

In [67]:
rng = np.random.default_rng()

def compute_pi_cpu(n_points):
    #First, generate random points
    rng = np.random.RandomState(42)
    x_rand = rng.random(n_points)
    y_rand = rng.random(n_points)

    #Compute radius from origin
    inside = np.sqrt(x_rand**2+y_rand**2) <= 1.0
    #Count number of points inside
    n_inside = np.sum(inside)

    #n_inside = 0
    #for i in range(n_points):
    #    n_inside += np.sqrt(x_rand[i]**2+y_rand[i]**2) <= 1.0

    #We can estimate pi by the following formula:
    #pi = 4 * n_inside / n_total
    pi = 4*n_inside/n_points

    return pi

In [70]:
tic = time.time()
print(compute_pi_cpu(5120000))
toc = time.time()

#for loop: 1.84 seconds for 512 000 elements
#vectorized: 0.018 seconds for 512 000 elements => 100 times faster!!!

print("Time taken: {:f} seconds".format(toc-tic))

3.1418
Time taken: 0.141325 seconds


In [54]:
pi_kernel_src = """
//Based on Stroustrup, adapted for CUDA
//pseudorandom numbers
__device__ float generateRandomNumber(long& last_draw) {
    last_draw = last_draw*1103515245 + 12345;
    long abs = last_draw & 0x7fffffff;
    return abs / 2147483648.0;
}


__global__ void computePi(unsigned int* inside, unsigned int num_iterations, unsigned int seed) {
    __shared__ unsigned int inside_shared[512];

    unsigned int tid = threadIdx.x;
    unsigned int bid = blockIdx.x;

    //1 generate random numbers
    unsigned int num_inside = 0;
    for (int i=0; i<num_iterations; ++i) {
        long rand_seed = seed + blockIdx.x*blockDim.x + threadIdx.x;
        float x = generateRandomNumber(rand_seed);
        float y = generateRandomNumber(rand_seed);

        //2 compute radius from origin
        float r = sqrt(x*x+y*y);

        //3 check if inside circle and write to memory
        if (r <= 1) {
            num_inside += 1;
        }
    }
    inside_shared[tid] = num_inside;

    /////////////////////////
    //Shared memory reduction
    /////////////////////////

    // Synchronze so that all thread see the same shared memory
    __syncthreads();

    // Find the sum in shared memory
    //Reduce from 512 to 256 elements
    if (threadIdx.x < 256) {
        inside_shared[threadIdx.x] = inside_shared[threadIdx.x] + inside_shared[threadIdx.x + 256];
    }
    __syncthreads();

    //Reduce from 256 to 128 elements
    if (threadIdx.x < 128) {
        inside_shared[threadIdx.x] = inside_shared[threadIdx.x] + inside_shared[threadIdx.x + 128];
    }
    __syncthreads();

    //Reduce from 128 to 64 elements
    if (threadIdx.x < 64) {
        inside_shared[threadIdx.x] = inside_shared[threadIdx.x] + inside_shared[threadIdx.x + 64];
    }
    __syncthreads();

    //Reduce from 32 to 16 elements
    //Since we here have only one active warp (threadIdx.x > 32)
    //we do not need to call syncthreads anymore
    volatile unsigned int* p = &inside_shared[0]; //To help the compiler not cache this variable...
    if (threadIdx.x < 32) {
        p[threadIdx.x] = p[threadIdx.x] + p[threadIdx.x + 32];
        p[threadIdx.x] = p[threadIdx.x] + p[threadIdx.x + 16];
        p[threadIdx.x] = p[threadIdx.x] + p[threadIdx.x + 8];
        p[threadIdx.x] = p[threadIdx.x] + p[threadIdx.x + 4];
        p[threadIdx.x] = p[threadIdx.x] + p[threadIdx.x + 2];
        p[threadIdx.x] = p[threadIdx.x] + p[threadIdx.x + 1];
    }

    // Finally write out to output
    // NOTE: We have 512 threads, but only thread 0 writes to memory
    if (threadIdx.x == 0) {
        inside[bid] = p[0];
    }
}
"""

mod = SourceModule(pi_kernel_src)
compute_pi_gpu_kernel = mod.get_function("computePi")

In [43]:
#prepare a function call: give pycuda instructions
#on what type of parameters to send to CUDA
#__global__ void computePi(
# unsigned int* inside, <= P
# unsigned int num_iterations, <= i
# unsigned int seed <= i
# );
compute_pi_gpu_kernel.prepare("Pii")

In [48]:
def compute_pi_gpu(n_points, iterations_per_thread=1000, threads_per_block=512):
    #10 000 000 points gave 3.141932
    #100 000 000 points gave 3.14141676
    #n_total = 1000000

    assert(n_points % (threads_per_block*iterations_per_thread) == 0)

    #Allocate output data on the GPU
    #Bytes per unsigned int:
    bytes_per_uint = 4
    inside_gpu = cuda.mem_alloc(bytes_per_uint*(n_points//threads_per_block)//iterations_per_thread)

    #Execute the pi-kernel
    num_blocks = (n_points // threads_per_block)//iterations_per_thread
    block=(threads_per_block,1,1)
    grid=(num_blocks,1,1)
    #compute_pi_gpu_kernel(inside_gpu, np.uint32(iterations_per_thread), np.uint32(time.time()), block=(threads_per_block,1,1), grid=(num_blocks,1,1))
    compute_pi_gpu_kernel.prepared_call(grid, block, inside_gpu, np.uint32(iterations_per_thread), np.uint32(42.0))#np.uint32(time.time()))

    #Allocate memory to download to on the CPU
    inside_cpu = np.empty((n_points//threads_per_block)//iterations_per_thread, dtype=np.uint32)

    #Download from the GPU to the CPU
    cuda.memcpy_dtoh(inside_cpu, inside_gpu)

    #Count number of points inside
    # Version 6: move this reduction to the GPU, and only transfer a single number to the CPU.
    n_inside = np.sum(inside_cpu)

    #We can estimate pi by the following formula:
    pi = 4*n_inside/n_points

    return pi

3.140625
Time taken: 0.002046 seconds


In [44]:
tic = time.time()
print(compute_pi_gpu(5120000, 10000, 512))
toc = time.time()

print("Time to execute gpu version: {:f} seconds".format(toc-tic))